In [1]:
import pandas as pd

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
system_content = [
    'Create a step-by-step plan for finding the answer to the following problem. Do not perform the actions in the plan.',
    'Now perform the steps in the plan you created using the most accurate and up-to-date data available. Balance detail and brevity in your response.',
]
temperature = 0.2
dataset = pd.read_json('../resources/data/Franklin/full_study.jsonl', lines=True)
data = dataset[['internal_id', 'question']]